# PLN Backward Chaining

*Backward Chaining* is (or planned to be) used in *OpenCog* for :<br>
> * *[OpenPsi](https://wiki.opencog.org/w/OpenPsi) planning*<br>
> * improving fitness estimates of *[MOSES](https://wiki.opencog.org/w/Meta-Optimizing_Semantic_Evolutionary_Search)* models<br>
> * generally, for finding subtler patterns that specialized algorithms like the *pattern miner* miss.

For a Background on *PLN* see [Probabilistic logic networks](https://wiki.opencog.org/w/Probabilistic_logic_networks)

# Theory

In simple rule-based systems, there are two kinds of inference, [forward chaining](https://wiki.opencog.org/w/Unified_rule_engine#Forward_Chainer) and [backward chaining](https://wiki.opencog.org/w/Unified_rule_engine#Backward_Chainer).

**Backward chaining :**  given a target, an atom that may or may not contain free variables, the system tries to find an inference tree that proves the target, grounding the free variables if necessary.<br>
Because the target determines which rules are selected and used, this method is considered *goal-driven*, in contrast to *data-driven* *forward-chaining* inference. Backward chaining is also considered to be *goal-directed* inference, or *hypothesis driven*, because inferences are not performed until the system is made to prove a particular goal (i.e. a question or target).<br>
*Backward Chaining* is used when you want to work backwards from the [Consequent](http://en.wikipedia.org/wiki/Consequent) to the [Antecedent](http://en.wikipedia.org/wiki/Antecedent_(logic%29). It is usually the part that follows *'then'* in a proposition.

**Examples of antecedents & consequents :**
> -  If **_P_**, then **_Q_**.<br>
**_Q_** is the consequent of this hypothetical proposition. **_P_** is the antecedent.<br>
-  If **_X_** is a mammal, then **_X_** is an animal.<br>
Here, " **_X_** is an animal" is the consequent, and " **_X_** is a mammal" an antecedent.<br>
-  If OpenCog can think, then the Singularity is near.<br>
"the Singularity is near" is the consequent, and "OpenCog can think" is an antecedent.<br>

**A logical example of Backward Chaining**<br>
> Here we will try to satisfy the goal of determining whether 'Plato is mortal' or not.<br>
Known facts:<br>
> -  Plato is a philosopher.
> -  Plato likes to wrestle.<br>

> The goal is to find out whether Plato is mortal based on a rule base : <br>
> 1. **If** X is a philosopher and likes to wrestle **then** X is a man.
> 2. **If** X can fly and lives in the sky **then** X is a god.
> 3. **If** X is a god **then** X is immortal.
> 4. **If** X is a man **then** X is mortal. <br>

> Using backward reasoning we can determine whether Plato is mortal in 4 steps. As we can see from rule 4, that this rule has a consequent of 'X is mortal' matches our goal to find out if 'Plato is mortal', so this checks out. Look at the antecedent to step 4 (the bit just after the If clause) - it is 'X is a man'. So now this antecedent (Substituting X for Plato) becomes the new goal of finding out if 'Plato is a man'.<br> 
We can see rule 1 contains the consequent 'X is a man', and can see the antecedents are .. 'X is a philosopher', and 'X likes to wrestle'.<br>
So now the antecedent(s) are broken down into two subgoals...<br> 
Given the two facts of 'Plato is a Philosopher' and 'Plato likes to wrestle', we can see that these two subgoals are satisfied.<br> 
We find rule 1 matches - and find that it is true that the consequent of rule 4 'X is mortal' matches - and therefore Plato is indeed mortal.

**More theory**<br>
- Here is short explainer of [Backward Chaining by Jocelyn Ireson-Paine](http://www.j-paine.org/students/lectures/lect3/node12.html)<br>
- Also see the [Wikipedia Page on Backward Chaining](http://en.wikipedia.org/wiki/Backward_chaining)

# Introduction
This tutorial can be practiced both on the jupyter notebook and the scheme environment.<br>
For the case of scheme environment, you need to copy-paste the codes into the scheme shell.<br>
For the case of jupyter notebook, if you have installed [Guile kernel](https://github.com/jerry40/guile-kernel), you can simply run the code cells and observe the output.<br>

**NOTE :** You need to follow the sub-sections listed below only if you are practicing using the jupyter notebook. <br>
After completing them, move to [**"Back after loading the files"**](#Back-after-loading-the-files) sub-section.<br>
- [**_load the code of knowledge-base.scm_**](#load-the-code-of-knowledge-base.scm)
- [**_load the code of conditional-instantiation-meta-rule.scm_**](#load-the-code-of-conditional-instantiation-meta-rule.scm)
- [**_load the code of fuzzy-conjunction-introduction-rule.scm_**](#load-the-code-of-fuzzy-conjunction-introduction-rule.scm)
- [**_load the code of rule-base.scm_**](#load-the-code-of-rule-base.scm)

For those practicing on the scheme shell don't follow these subsections, since they are the equivalent for [_**`[1]`**_](#on-scheme-shell-:). Instead jump directly to [**"Back after loading the files"**](#Back-after-loading-the-files) sub-section.


# Practice
The following example is a simplified version of *[frog](https://github.com/opencog/atomspace/tree/master/examples/rule-engine/frog)* and relies on some of its files. <br>
So for practice on the scheme shell, open *terminal* and go under :

> _$ cd atomspace/examples/rule-engine/frog_

and fire up guile
> _$ guile_

then copy-paste the codes.

#### **_load required opencog modules_**
set the logging level to _debug_ (will be handy to understand what the backward chainer does)


In [1]:
(use-modules (ice-9 readline)) 
(activate-readline)
(add-to-load-path "/usr/local/share/opencog/scm")
(add-to-load-path ".")
(use-modules (opencog))
(use-modules (opencog query))
(use-modules (opencog exec))
(use-modules (opencog logger))
(cog-logger-set-level! "debug")

"INFO"

In this example we have a black box for which we know there is something in it.<br>
> Known facts :<br>
- The thing in the box makes croaking sounds.
- The thing in the box eats flies.<br>

> Goal :<br>
- Find whether the color of the thing in the black box is green.<br>

> We have the following relations (rules) defined :<br>
- If X croaks and X eats flies, then X is a frog.
- If X is a frog, then X is green.<br>

Let's say the thing in the black box is named _Fritz_ and from the above relations we need to deduce its color.<br>
Now load the *rule-engine* module, as well as the *[knowledge-base](https://github.com/opencog/atomspace/blob/master/examples/rule-engine/frog/knowledge-base.scm)* and *[rule-base](https://github.com/opencog/atomspace/blob/master/examples/rule-engine/frog/rule-base.scm)*

**_load the rule-engine module_**

In [3]:
(use-modules (opencog rule-engine))

### **_load the knowledge-base and the rule-base scripts_**

#### on scheme shell :

> _scheme@(guile-user)> (load "knowledge-base.scm")_

> _scheme@(guile-user)> (load "rule-base.scm")_


**Here are the file contents**<br>
run the cells bellow to load the files into the notebook environment.<br> 

#### **_load the code of knowledge-base.scm_**

In [4]:
;;;;;;;;;;;;;;;;;;;;
;; Knowledge base ;;
;;;;;;;;;;;;;;;;;;;;

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (And
      (Evaluation
         (Predicate "croaks")
         (Variable "$X"))
      (Evaluation
         (Predicate "eats_flies")
         (Variable "$X")))
   (Inheritance
      (Variable "$X")
      (Concept "frog")))

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (And
      (Evaluation
         (Predicate "chirps")
         (Variable "$X"))
      (Evaluation
         (Predicate "sings")
         (Variable "$X")))
   (Inheritance
      (Variable "$X")
      (Concept "Canary")))

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (Inheritance
      (Variable "$X")
      (Concept "frog"))
   (Inheritance
      (Variable "$X")
      (Concept "green")))

(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (Inheritance
      (Variable "$X")
      (Concept "Canary"))
   (Inheritance
      (Variable "$X")
      (Concept "yellow")))

(Evaluation (stv 1.0 1.0)
   (Predicate "croaks")
   (Concept "Fritz"))

(Evaluation (stv 1.0 1.0)
   (Predicate "chirps")
   (Concept "Tweety"))

(Inheritance (stv 1.0 1.0)
   (Concept "Tweety")
   (Concept "yellow"))

(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies")
   (Concept "Tweety"))

(Evaluation (stv 1.0 1.0)
   (Predicate "eats_flies")
   (Concept "Fritz"))

(EvaluationLink (stv 1 1)
   (PredicateNode "eats_flies")
   (ConceptNode "Fritz")
)


**NOTE :** The following two are files, *conditional-instantiation-meta-rule.scm* and *fuzzy-conjunction-introduction-rule.scm*, which are required by *rule-base.scm*.<br> 

#### **_load the code of conditional-instantiation-meta-rule.scm_**

In [5]:
;; =======================================================================
;; Conditional Instantiation Meta Rule
;;
;; ImplicationScopeLink
;;    V
;;    P
;;    Q
;; |-
;;    T
;;    P
;;    |-
;;    Q[V->T]
;;
;; where V is a variable or a list of variables, P is a condition, Q
;; is the implicand, T is an atom (or a list of atoms) satisfying the
;; V type constraints, and such that P[V->T], P where V has been
;; subtituted by T has a meaningful TV, and Q[V->T] is Q where V has
;; been substituted by T.
;; -----------------------------------------------------------------------

(use-modules (srfi srfi-1))

(use-modules (opencog exec))
(use-modules (opencog logger))

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;; Implication full instantiation rule ;;
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

(define conditional-full-instantiation-meta-variables
  (VariableList
     (TypedVariable
        (Variable "$TyVs")
        (TypeChoice
           (Type "TypedVariableLink")
           (Type "VariableList")))
     (Variable "$P")
     (Variable "$Q")))

(define conditional-full-instantiation-meta-body
  (let* ((implication (Quote (ImplicationScope
                         (Unquote (Variable "$TyVs"))
                         (Unquote (Variable "$P"))
                         (Unquote (Variable "$Q")))))
         (precondition (Evaluation
                         (GroundedPredicate "scm: true-enough")
                         implication)))
  (And
    implication
    precondition)))

;; Here only the implicant is considered as premise. The variable(s)
;; should as well so the backward chainer can consider them as
;; targets, however that implies to lay them out explicitly, so for
;; sake of simplicity they are ignored for now.
(define conditional-full-instantiation-meta-rewrite
  (let* ((TyVs (Variable "$TyVs"))
         (P (Variable "$P"))
         (Q (Variable "$Q"))
         (implication (Quote (ImplicationScope
                         (Unquote TyVs)
                         (Unquote P)
                         (Unquote Q)))))
    (Quote (Bind
      (Unquote TyVs)
      (And
        (Unquote (LocalQuote (LocalQuote P)))
        (Evaluation (GroundedPredicate "scm: true-enough") (Unquote P)))
      (ExecutionOutput
        (GroundedSchema "scm: conditional-full-instantiation-formula")
        (Unquote
          (ListLink
            Q
            implication
            P)))))))

(define conditional-full-instantiation-meta-rule
  (BindLink
     conditional-full-instantiation-meta-variables
     conditional-full-instantiation-meta-body
     conditional-full-instantiation-meta-rewrite))

;; Return the TV of the evaluation of `an` assuming that it is an
;; AndLink, that is return the TV which is the the min over the
;; strengths and confidences of its outgoings. Normally this would be
;; handled by a rule but to simplify the unit test it is hardcored
;; here.
(define (conjunction-fuzzy-eval an)
  (let* ((outg (cog-outgoing-set an))
         (min-s-atom (min-element-by-key outg cog-stv-strength))
         (min-c-atom (min-element-by-key outg cog-stv-confidence))
         (min-s (cog-stv-strength min-s-atom))
         (min-c (cog-stv-confidence min-s-atom)))
    (stv min-s min-c)))

(define (true-enough-bool a)
  (let ((s (cog-stv-strength a)) (c (cog-stv-confidence a)))
    (and (> s 0.5) (> c 0.5))))

(define (true-enough a)
  (bool->tv (true-enough-bool a)))

;; Set (stv 1 1) on Q is Impl and P strength are both above 0.5 and
;; their confidence is non null.
(define (conditional-full-instantiation-formula Q Impl P)
  ;; Evaluate Q
  (if (and (true-enough-bool Impl) (true-enough-bool P))
      (cog-set-tv! Q (stv 1 1))))

;; Name the meta rule
(define conditional-full-instantiation-meta-rule-name
  (DefinedSchemaNode "conditional-full-instantiation-meta-rule"))
(DefineLink conditional-full-instantiation-meta-rule-name
  conditional-full-instantiation-meta-rule)


(DefineLink
   (DefinedSchemaNode "conditional-full-instantiation-meta-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$TyVs")
            (TypeChoice
               (TypeNode "TypedVariableLink")
               (TypeNode "VariableList")
            )
         )
         (VariableNode "$P")
         (VariableNode "$Q")
      )
      (AndLink
         (EvaluationLink
            (GroundedPredicateNode "scm: true-enough")
            (QuoteLink
               (ImplicationScopeLink
                  (UnquoteLink
                     (VariableNode "$TyVs")
                  )
                  (UnquoteLink
                     (VariableNode "$P")
                  )
                  (UnquoteLink
                     (VariableNode "$Q")
                  )
               )
            )
         )
         (QuoteLink
            (ImplicationScopeLink
               (UnquoteLink
                  (VariableNode "$TyVs")
               )
        

#### **_load the code of fuzzy-conjunction-introduction-rule.scm_**

In [6]:
; =============================================================================
; Fuzzy conjunction introduction rule
;
; A1
; ...
; An
; |-
; AndLink
;   A1
;   ...
;   An
;
; Where A1 to An are atoms with a fuzzy TV
; -----------------------------------------------------------------------------

(use-modules (opencog rule-engine))
(use-modules (srfi srfi-1))

;; Generate variable (Variable prefix + "-" + to_string(i))
(define (gen-variable prefix i)
  (Variable (string-append prefix "-" (number->string i))))

;; Generate a list of variables (Variable prefix + "-" + to_string(n))
(define (gen-variables prefix n)
  (if (= n 0)
      ;; Base case
      '()
      ;; Recursive case
      (append (gen-variables prefix (- n 1))
              (list (gen-variable prefix (- n 1))))))

;; Generate a fuzzy conjunction introduction rule for an n-ary
;; conjunction
(define (gen-fuzzy-conjunction-introduction-rule nary)
  (let* ((variables (gen-variables "$X" nary))
         (EvaluationT (Type "EvaluationLink"))
         (InheritanceT (Type "InheritanceLink"))
         (type (TypeChoice EvaluationT InheritanceT))
         (gen-typed-variable (lambda (x) (TypedVariable x type)))
         (vardecl (VariableList (map gen-typed-variable variables)))
         (pattern (And variables))
         (rewrite (ExecutionOutput
                    (GroundedSchema "scm: fuzzy-conjunction-introduction-formula")
                    ;; We wrap the variables in Set because the order
                    ;; doesn't matter and that way alpha-conversion
                    ;; works better.
                    (List (And variables) (Set variables)))))
    (Bind
      vardecl
      pattern
      rewrite)))

;; Return true if all elements of the list are unique
(define (is-set l)
  (cond ((null? l) #t)
        ((member (car l) (cdr l)) #f)
        (else (is-set (cdr l)))))

;; Check that they all are different, and have positive confidences
(define (fuzzy-conjunction-introduction-precondition S)
  (bool->tv (is-confident-enough-set (cog-outgoing-set S))))

(define (is-confident-enough-set andees)
  (let* ((confident-enough (lambda (A) (> (cog-stv-confidence A) 0))))
    (and (is-set andees)
         (every confident-enough andees))))

(define (fuzzy-conjunction-introduction-formula A S)
  (let* ((andees (cog-outgoing-set S))
         (min-s-atom (min-element-by-key andees cog-stv-strength))
         (min-c-atom (min-element-by-key andees cog-stv-confidence))
         (min-s (cog-stv-strength min-s-atom))
         (min-c (cog-stv-confidence min-c-atom)))
    (if (is-confident-enough-set andees)       ; only introduce meaningful
                                               ; conjunction of unique andees
        (cog-set-tv! A (stv min-s min-c)))))

;; Name the rules
(define fuzzy-conjunction-introduction-2ary-rule
  (gen-fuzzy-conjunction-introduction-rule 2))
(define fuzzy-conjunction-introduction-2ary-rule-name
  (DefinedSchema "fuzzy-conjunction-introduction-2ary-rule"))
(DefineLink
  fuzzy-conjunction-introduction-2ary-rule-name
  fuzzy-conjunction-introduction-2ary-rule)


(DefineLink
   (DefinedSchemaNode "fuzzy-conjunction-introduction-2ary-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$X-0")
            (TypeChoice
               (TypeNode "EvaluationLink")
               (TypeNode "InheritanceLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$X-1")
            (TypeChoice
               (TypeNode "EvaluationLink")
               (TypeNode "InheritanceLink")
            )
         )
      )
      (AndLink
         (VariableNode "$X-0")
         (VariableNode "$X-1")
      )
      (ExecutionOutputLink
         (GroundedSchemaNode "scm: fuzzy-conjunction-introduction-formula")
         (ListLink
            (AndLink
               (VariableNode "$X-0")
               (VariableNode "$X-1")
            )
            (SetLink
               (VariableNode "$X-0")
               (VariableNode "$X-1")
            )
         )
      )
   )
)


#### **_load the code of rule-base.scm_**

In [7]:
;;;;;;;;;;;;;;;
;; Rule base ;;
;;;;;;;;;;;;;;;
(define ci-rbs (ConceptNode "ci-rbs"))
(Inheritance ci-rbs (ConceptNode "URE"))

;; Associate the rules to the rule base (with weights, their semantics
;; is currently undefined, we might settled with probabilities but it's
;; not sure)
(MemberLink (stv 1 1)
   conditional-full-instantiation-meta-rule-name
   ci-rbs
)
(MemberLink (stv 1 1)
   fuzzy-conjunction-introduction-2ary-rule-name
   ci-rbs
)

;; termination criteria parameters
(ExecutionLink
   (SchemaNode "URE:maximum-iterations")
   ci-rbs
   (NumberNode "20")
)

;; Attention allocation (set the TV strength to 0 to disable it, 1 to
;; enable it)
(EvaluationLink (stv 0 1)
   (PredicateNode "URE:attention-allocation")
   ci-rbs
)

(EvaluationLink (stv 0 1)
   (PredicateNode "URE:attention-allocation")
   (ConceptNode "ci-rbs")
)


#### Back after loading the files

In knowledge-base.scm. You may observe [ImplicationScopeLinks](https://wiki.opencog.org/w/ImplicationScopeLink) such as<br>
> *(ImplicationScope (stv 1.0 1.0)
   (TypedVariable
      (Variable "$X")
      (Type "ConceptNode"))
   (And
      (Evaluation
         (Predicate "croaks")
         (Variable "$X"))
      (Evaluation
        (Predicate "eats_flies")
        (Variable "$X")))
   (Inheritance
      (Variable "$X")
      (Concept "frog")))*<br><br>
**encoding the relationship :**<br>
If X croaks and X eats flies - Then X is a frog

At the end of that same file, there are facts regarding *Fritz* and other animals as they relate to attributes using [EvaluationLinks](https://wiki.opencog.org/w/EvaluationLink).
You may want to take a look at *rule-base.scm* which loads rules, associate them to a rule-base (called *ci-rbs*) and set some *URE* parameters (such as *URE:maximum-iterations*). Explore the rule files *[conditional-instantiation-meta-rule.scm](https://github.com/opencog/atomspace/blob/master/examples/rule-engine/meta-rules/conditional-instantiation-meta-rule.scm)* and *[fuzzy-conjunction-introduction-rule.scm](https://github.com/opencog/atomspace/blob/master/examples/rule-engine/rules/fuzzy-conjunction-introduction-rule.scm)* as well. Don't worry if you don't understand it, just read the comments on the top of the files that explains what the rules are doing.<br>
What the backward chainer is gonna do is use these rules over the knowledge base to find out what is *green*.

We are going to call the backward chainer but let's first define the target, meaning we want atoms that have an inheritance relationship with the *ConceptNode "green"*.

In [8]:
;define target, which is "green" in our case
(define target
  (InheritanceLink (VariableNode "$what") (ConceptNode "green"))
)

To simplify the life of the backward chainer and avoid nonsensical answers (just as the target itself), let's restrict the type of the variable (VariableNode "$what")

In [9]:
;restrict the type of the variable (VariableNode "$what") to ConceptedNode type
(define vardecl
  (TypedVariable (VariableNode "$what") (TypeNode "ConceptNode"))
)

In [10]:
;calling the backward chainer
(cog-bc ci-rbs target #:vardecl vardecl)

(SetLink
   (InheritanceLink (stv 1 1)
      (ConceptNode "Fritz")
      (ConceptNode "green")
   )
)


**Output :** <br>
(SetLink <br>
   (InheritanceLink (stv 1 1)<br>
      (ConceptNode "Fritz")<br>
      (ConceptNode "green")<br>
   )<br>
)

If you wish to understand in more depth how the backward chainer came up with the answer, open the log file *opencog.log* under the same directory. Search the messages logged by the URE components (containing the string "[URE]") and try to make sense of what you see. In particular you may find that the inference trees evolved by the backward chainer are logged right after the message with inference tree. For instance the inference tree corresponding to the solution is :

                          [12543950459016998847][1] [15852562040974555944][1]
                          ======fuzzy-conjunction-introduction-formula=======
             [14398977004689637744][1] [15383395919935236078][1]
             ------conditional-full-instantiation-formula-------
[15603332628456784367][1] [13829542973489658864][1]<br>
------conditional-full-instantiation-formula-------<br>
             [13520875333812722983][1]<br>
where the numbers corresponds to the UUIDs of the atoms, target, intermediary targets and premises, involved with the inference. If you search these UUIDs you can get the corresponding atoms in the log file.

* **If you dear reader think of some good quiz questions, add them to the discussion branch of this wiki page**